In [ ]:
import spikeinterface.full as si
import glob
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from probeinterface.plotting import plot_probe
%matplotlib qt 

# Paths
base_folder = Path('/media/e4/data1/RomansData/bird1/') 
folders = sorted(glob.glob((base_folder / 'Rec_*_g0').as_posix()))

In [ ]:
folders

In [ ]:
folder= folders[6]
print(folder)
output_folder = Path(folder) / 'by_channel_data'
output_folder.mkdir(exist_ok=True, parents=True)

In [ ]:
rec_lf = si.SpikeGLXRecordingExtractor(folder, stream_id='imec0.lf') # LF
print(rec_lf)

In [ ]:
rec_lf.channel_ids

In [ ]:
# Bird1 : LMAN 'safe' channels: 202-236; AreaX: 50-176 (Nchan slightly overestimating)
areaX_channels = rec_lf.channel_ids[50:190][::10] # to make multiples of 10
lman_channels = rec_lf.channel_ids[202:252][::10]  # to make multiples of 10
unit_related_channels = [
'imec0.lf#LF191',
'imec0.lf#LF230',
'imec0.lf#LF222',
] # got this from filter+cmr_radius/tridesclous/

selection_list = np.concatenate([areaX_channels, lman_channels, unit_related_channels])

In [ ]:
selection_list = sorted(np.unique(selection_list))
print(selection_list)

In [ ]:
sliced_channels = rec_lf.channel_slice(channel_ids=selection_list)
traces_channels = sliced_channels.get_traces(return_scaled=True)
transposed_traces = traces_channels.T

In [ ]:
# this should be true
len(traces_channels[:,0])/rec_lf.get_sampling_frequency() == rec_lf.get_total_duration()

In [ ]:
transposed_traces

In [ ]:
# this should be True
sliced_channels.get_channel_ids() == selection_list

In [ ]:
for trace_indx in range(len(selection_list)):
    name_channel = selection_list[trace_indx]
    print(name_channel)
    trace = transposed_traces[trace_indx]
    print(trace)
    name_channel = selection_list[trace_indx]
    np.save(output_folder / (name_channel + '.npy'), trace)

## In case you need to check probe

In [ ]:
rec_ap = si.SpikeGLXRecordingExtractor(folder, stream_id='imec0.ap') # microphone

In [ ]:
rec_lf.set_probe(rec_ap.get_probe(), in_place=True)

In [ ]:
def check_probe_channels(rec):
    probe = rec.get_probe()
    plot_probe(probe, show_channel_on_click=True)
    plt.show()

check_probe_channels(rec_lf)

In [ ]:
location = rec_lf.get_channel_locations()
plt.scatter(location[:,0], location[:,1])
plt.show()

## Get spiketrains

In [ ]:
sorting_path = Path('/media/e4/data1/RomansData/bird1/Rec_3_19_11_2021_g0/sorting_20220116/full/filter+cmr_radius/tridesclous/custom_tdc_1/')

In [ ]:
sorting_tdc = si.TridesclousSortingExtractor(sorting_path)
print(sorting_tdc)

In [ ]:
unit_list = sorting_tdc.unit_ids

In [ ]:
for unit in unit_list:
    unit_spks = sorting_tdc.get_unit_spike_train(unit_id=unit, return_times=True)
    unit_name = 'unit_'+ str(unit)
    np.savetxt(output_folder/(unit_name+'.txt'), unit_spks)